# Call center case - Post-call transcription and analysis with Azure OpenAI Service

In this exercise, we will perform sentiment analysis and summerization using call center transcriptions. We will transribe the customer recording to text, then use OpenAI to detect sentiment. We also use OpenAI to summerize long text into a few sentences for further analysis.

In [1]:
from dotenv import load_dotenv
load_dotenv('environment_variables.env')

True

In [7]:
# from azure.ai.textanalytics import TextAnalyticsClient
# from azure.core.credentials import AzureKeyCredential
import azure.cognitiveservices.speech as speechsdk

import json, os
import string
import time
import wave

import openai
from openai import AzureOpenAI
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
# from openai.embeddings_utils import get_embedding, cosine_similarity
# from transformers import GPT2TokenizerFast


client = AzureOpenAI(
  azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT_TEXT"), 
  api_key=os.environ.get("AZURE_OPENAI_KEY_TEXT"),  
  api_version=os.environ.get("AZURE_OPENAI_API_VERSION")
)

SPEECH_KEY = os.environ.get("SPEECH_API_KEY")

COMPLETIONS_MODEL = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT")

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

def cosine_similarity(query_embedding, embeddings, distance_metric='cosine'):
    if distance_metric == 'cosine':
        distances = np.dot(embeddings, query_embedding) / (np.linalg.norm(embeddings) * np.linalg.norm(query_embedding))
        distances = 1 - distances  
    else:
        raise ValueError("Metrik (fungsi jarak) tidak didukung. Gunakan 'cosine'.")

    return distances

def recognize_speech_from_file(filename):
    # Set up the subscription info for the Speech Service:
    # Replace with your own subscription key and service region (e.g., "westus").
    speech_key = SPEECH_KEY
    service_region = "australiaeast"

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    audio_config = speechsdk.audio.AudioConfig(filename=filename)
    # Creates a speech recognizer using a file as audio input, also specify the speech language
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,  audio_config=audio_config)

    global done 
    done = False
    global recognized_text_list 
    recognized_text_list=[]
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        global done
        done = True

    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)
        # print('RECOGNIZED: {}'.format(evt.result.text))

    # Connect callbacks to the events fired by the speech recognizer
    # speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(lambda evt: print('STT SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('STT SESSION STOPPED {}'.format(evt)))
    # speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    # speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()

    return recognized_text_list

# Sentiment Analysis
### Transcribe Customer Call to Text

In [5]:
text = recognize_speech_from_file("../data/good_review.wav")
print(text)

STT SESSION STARTED: SessionEventArgs(session_id=b16eb40f8dbf4bca8ad3e7896641435d)
STT SESSION STOPPED SessionEventArgs(session_id=b16eb40f8dbf4bca8ad3e7896641435d)
CLOSING on SessionEventArgs(session_id=b16eb40f8dbf4bca8ad3e7896641435d)
['I had purchased another thermostat from a big box store and the Rep there assured me that it would turn off when it got too cold.', 'Welcome to find out. It did not and my heater in the garage was running a lot of the time.', 'This one was great and was easily wired.', 'It does turn off when you turn it all the way to the left.', "It's annoying they don't have any in Celsius, but whatever.", 'It runs well and actually turns off my garage heater.', 'Would totally purchase again.', 'Installed with five kilowatts heater in garage.', 'Thermostat failed an on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']


### Create Promot for sentiment analysis
Use natural language to instruct OpenAI to detect customer's sentiment

In [8]:
prompt = f"Detect whether customer is positive or negative.  Just say positive or negative.\n\n{' '.join(text)}"


response = client.chat.completions.create(
  model=COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])
print(response.choices[0].message.content)


Positive


### Use a negative example

In [9]:
text = recognize_speech_from_file("../data/bad_review.wav")
print(text)
prompt = f"Detect whether customer is positive or negative. Just say positive or negative.\n\n{' '.join(text)}"

response = client.chat.completions.create(
  model=COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])
print(response.choices[0].message.content)

STT SESSION STARTED: SessionEventArgs(session_id=49a202445c44483ab3ea497aefbb9d3b)
STT SESSION STOPPED SessionEventArgs(session_id=49a202445c44483ab3ea497aefbb9d3b)
CLOSING on SessionEventArgs(session_id=49a202445c44483ab3ea497aefbb9d3b)
['Installed with five kW heater in garage. Thermostat failed in on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']
Negative.


# Summarization

Use OpenAI to summerize customer message

In [10]:
text = recognize_speech_from_file("../data/good_review.wav")
print(text)
prompt = f"Summarize the following text.\n\n{' '.join(text)}"

response = client.chat.completions.create(
  model=COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])
print(response.choices[0].message.content)

STT SESSION STARTED: SessionEventArgs(session_id=0da337678d46460da64546e44fb32e18)
STT SESSION STOPPED SessionEventArgs(session_id=0da337678d46460da64546e44fb32e18)
CLOSING on SessionEventArgs(session_id=0da337678d46460da64546e44fb32e18)
['I had purchased another thermostat from a big box store and the Rep there assured me that it would turn off when it got too cold.', 'Welcome to find out. It did not and my heater in the garage was running a lot of the time.', 'This one was great and was easily wired.', 'It does turn off when you turn it all the way to the left.', "It's annoying they don't have any in Celsius, but whatever.", 'It runs well and actually turns off my garage heater.', 'Would totally purchase again.', 'Installed with five kilowatts heater in garage.', 'Thermostat failed an on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']
The person bought a thermostat from a big box store and was assured 

In [11]:
print(response.choices[0].message.content)

The person bought a thermostat from a big box store and was assured by the representative that it would turn off when it got too cold. However, the thermostat did not do this and the person's garage heater ran constantly. They found another thermostat that worked well and easily wired it. The only annoyance was that it did not have Celsius units. However, the thermostat eventually failed and stayed in the "on" position overnight, causing overheated connections that almost led to a fire. The thermostat was installed by a certified electrician. See attached photos.
